<a href="https://colab.research.google.com/github/Pager07/fruit-classifier/blob/master/FruitClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Github 

In [9]:
from google.colab import drive 
drive.mount('/content/gdrive' , force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!git init '/content/gdrive/My Drive/age-classifier'

In [10]:
!git add  '/content/gdrive/My Drive/age-classifier/FruitClassifier.ipynb'

fatal: not a git repository (or any of the parent directories): .git


# Fruit Classifier 

In [0]:
!curl -s https://course.fast.ai/setup/colab | bash

In [0]:
%reload_ext autoreload 
%autoreload 2
%matplotlib inline 